# Keras and Tensorflow

NYC Resistor class, 2019-12-07

https://github.com/michiexile/tensorflow-class

wifi: NYCR24 password: clubmate

# Tensorflow

![](TF_FullColor_Horizontal.svg)

https://tensorflow.org

Google Brain's internal machine learning library. Open Sourced November 2015.

*Graph computation* paradigm (also Theano, CNTK, ...)

Python API

Tensorflow.js, TensorFlow Lite (Android/iOS), Tensor Processing Unit, R, C++, Haskell, Java, Go, Rust, ...

At the core:

1. Tensor handling library
2. Numerics abstraction layer
3. Deep Learning friendly interaction abstraction